In [1]:
from collections import deque
import random
from tqdm import tqdm
from time import sleep
import json
from time import sleep
import gym
import numpy as np
from src.utils import (
    get_env, get_avg_slowdown, get_sjf_action,
    get_possible_actions, finisihed_job_cnt)

from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
from src.models.pointer_network import pointer_networks

with open("configs/env.json", 'r') as f:
    config = json.load(f)

env = get_env("configs/env.json")

sds = []
cnt = []

observation = env.reset(seq_no=0 % 50)

discount = 1.00
max_job_length = config['max_job_length']
capa = config['n_resource_slot_capacities']
n_resources = len(config['n_resource_slot_capacities'])
input_size = max_job_length * n_resources + 1
state_size = np.prod(observation['machine'].shape)
embedding_size = hidden_size = 32
num_sample_batch = 5

ptr_net = pointer_networks(
    state_size, input_size, embedding_size, hidden_size,
    max_job_length=max_job_length)

adam = optim.RMSprop(ptr_net.parameters(), lr=0.001)
loss = 0
cnt = 0

In [2]:
def get_trajs(model, env, dg=False):
    cc = np.random.randint(0, 100)
    state = env.reset(seq_no=cc)
    ob_obs = []
    ac_actions = []
    _ys = []
    baseline = np.zeros(shape=(num_sample_batch, config['ep_force_stop']), dtype=np.float32)
    batch_size = 16
    ent_mean = []
    for sample_batch in range(num_sample_batch):
        obs = []
        actions = []
        rewards = []
        list_y = []
        state = env.reset(seq_no=cc)
        for t in range(config['ep_force_stop']):
            if dg:
                print(t)
            action, action_indices, ent = model.get_action(state)
            ent_mean.append(ent)
            state2, reward, done, info = env.step(action)
            obs.append(state)
            actions.append(action_indices)
            rewards.append(reward)
            if done:
                break
            state = state2
        disc_vec = np.array(rewards, dtype=float)
        for i in range(1, len(rewards)):
            disc_vec[i:] *= discount
        for i in range(len(rewards)):
            y_i = np.sum(disc_vec[i:]) / (discount ** i)
            list_y.append(y_i)
        _ys.append(list_y)
        baseline[sample_batch, :len(list_y)] = list_y
        ob_obs.append(obs)
        ac_actions.append(actions)

    baseline = np.mean(baseline, axis=0)
    advs = []
    for sb in range(num_sample_batch):
        ys = np.array(_ys[sb])
        ADV = ys - baseline[:len(ys)]
        advs.append(ADV)
    return [item for sublist in ob_obs for item in sublist], np.hstack(ac_actions), np.hstack(advs)

In [3]:
exe = ThreadPoolExecutor(max_workers=4)

In [4]:
envs = [get_env("configs/env.json") for i in range(4)]

In [ ]:
for iter in range(1000):
    adam.zero_grad()
    loss = 0
    cnt = 0
    fts = []
    for jj in range(4):
        fts.append(
            exe.submit(get_trajs, ptr_net, envs[jj]))
    loss = 0
    cnt = 0
    adam.zero_grad()
    for future in as_completed(fts):
        obs, acs, advs = future.result()
        for ob, ac, adv in zip(obs, acs, advs):
            if len(ac) > 0:
                t_loss = ptr_net.train_single(ob, ac, torch.as_tensor(adv, dtype=torch.float32))
                loss += t_loss
                cnt += 1
    loss /= cnt
    loss.backward()
    adam.step()
    print("loss : ", loss.detach().numpy())
    if (iter >= 0) and (iter % 5 == 0):
        usages = np.empty(shape=(0, 3))
        ep_lengths = []
        slowdowns = []
        rews = []
        ents = []
        fjc = []
        for i in range(30):
            s = env.reset(seq_no=100 + i)
            rr = []
            action = []
            for ep_len in range(config['ep_force_stop']):
                action, action_indices, entropy = ptr_net.get_action(s, argmax=True)
                usage = (100 - env.machine.avbl_slot[0]) / 100.0
                usages = np.vstack([usages, usage])
                s2, r, done, info = env.step(action)
                ents.append(entropy)
                rr.append(r)
                if done:
                    break
                s = s2
            fjc.append(finisihed_job_cnt(info))
            rews.append(np.mean(rr))
            slowdowns.append(get_avg_slowdown(info))

        print("[iter %d] avg rewards: %0.3f\tavg slowdowns: %0.3f, etnropies %0.2f, fjc: %d" %
                (iter, np.mean(rews), np.mean(slowdowns), np.mean(ents), np.sum(fjc)))
        print("avg usages", np.mean(usages, axis=0))

loss :  15.108337
[iter 0] avg rewards: -20.298	avg slowdowns: 8.735, etnropies 1.42, fjc: 10459
avg usages [0.55224726 0.56101313 0.56915755]
loss :  12.123537
loss :  12.659521
loss :  9.348535
loss :  15.87744
loss :  15.3166
[iter 5] avg rewards: -15.289	avg slowdowns: 6.547, etnropies 1.31, fjc: 10459
avg usages [0.54960148 0.55832535 0.56643075]
loss :  14.694913
loss :  24.230658
loss :  15.4521055
loss :  13.176153
loss :  8.291974
[iter 10] avg rewards: -14.390	avg slowdowns: 6.119, etnropies 1.12, fjc: 10459
avg usages [0.55116183 0.55991046 0.56803887]
loss :  3.924241
loss :  8.444166
loss :  2.4337926
loss :  7.280006
loss :  -3.9581423
[iter 15] avg rewards: -14.191	avg slowdowns: 6.079, etnropies 1.00, fjc: 10459
avg usages [0.54745553 0.55614534 0.56421909]
loss :  -7.932385
loss :  4.600731
loss :  1.5257144
loss :  -8.584892
loss :  -8.588123
[iter 20] avg rewards: -14.286	avg slowdowns: 6.099, etnropies 0.93, fjc: 10459
avg usages [0.54960148 0.55832535 0.56643075]
l